In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
tavily_api_key = os.getenv("TAVILY_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1732061823, 'localtime': '2024-11-19 16:17'}, 'current': {'last_updated_epoch': 1732061700, 'last_updated': '2024-11-19 16:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 243, 'wind_dir': 'WSW', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 53, 'cloud': 100, 'feelslike_c': 12.5, 'feelslike_f': 54.5, 'windchill_c': 11.2, 'windchill_f': 52.2, 'heatindex_c': 11.9, 'heatindex_f': 53.5, 'dewpoint_c': 7.1, 'dewpoint_f': 44.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.1, 'gust_mph': 8.7, 'gust_kph': 14.1}}"}, {'url': 'https://forecast.weather.gov/MapClick.p

In [6]:
tools

[TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]

In [5]:
"""
A Simple Agent using Cohere API
"""

from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

# Initialize the Cohere model
model = ChatCohere()

# Invoke the Cohere model with a simple prompt
response = model.invoke([HumanMessage(content="hi!")])
print(response.content)

# Example: Binding tools to the Cohere model
model_with_tools = model.bind_tools(tools)  # Bind the tools to the model

# Invoke the model with tools
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# Example of asking a tool-enabled question
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


Hello! How can I help you today?
ContentString: Hi there! How can I help you today?
ToolCalls: []
ContentString: I will search for 'weather in SF' to find the answer.
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'd3657af990174b41bc2fc8a4d8a7cfb4', 'type': 'tool_call'}]


In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

print(response["messages"])

response = agent_executor.invoke({"messages": [HumanMessage(content="WHAT IS THE WEATHER IN PUNE!")]})
print(response["messages"])

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='369a93a0-db1d-4850-9af0-27dd72cc8efb'), AIMessage(content='Hi! How can I help you today?', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'ad080a35-38a1-4875-8937-b45642f7a64a', 'token_count': {'input_tokens': 925.0, 'output_tokens': 80.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'ad080a35-38a1-4875-8937-b45642f7a64a', 'token_count': {'input_tokens': 925.0, 'output_tokens': 80.0}}, id='run-00918b5e-1a91-44b2-aeca-3d62c0f79b85-0', usage_metadata={'input_tokens': 925, 'output_tokens': 80, 'total_tokens': 1005})]
[HumanMessage(content='WHAT IS THE WEATHER IN PUNE!', additional_kwargs={}, response_metadata={}, id='4ebee13e-c92c-47f7-ba94-c2ace5466ad0'), AIMessage(content="I will search for 'weather in Pu

In [22]:
# response['messages'][3]

In [23]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "123"}}


for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I help you today?', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7b73c5a4-35dd-4638-a97a-29424ef30abd', 'token_count': {'input_tokens': 927.0, 'output_tokens': 82.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7b73c5a4-35dd-4638-a97a-29424ef30abd', 'token_count': {'input_tokens': 927.0, 'output_tokens': 82.0}}, id='run-a572770e-47d1-4206-bc57-dfa17eec34f7-0', usage_metadata={'input_tokens': 927, 'output_tokens': 82, 'total_tokens': 1009})]}}
----


In [26]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "123"}}


for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is my Name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '9f26978e-1560-410c-8647-0cc6d073adb5', 'token_count': {'input_tokens': 986.0, 'output_tokens': 60.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '9f26978e-1560-410c-8647-0cc6d073adb5', 'token_count': {'input_tokens': 986.0, 'output_tokens': 60.0}}, id='run-51104048-258d-4794-8875-3bb63a059f31-0', usage_metadata={'input_tokens': 986, 'output_tokens': 60, 'total_tokens': 1046})]}}
----
